In [99]:
from llama_index import LLMPredictor, ServiceContext
# from llama_index import VectorStoreIndex
# from llama_index import SimpleDirectoryReader
from llama_index import Prompt
from llama_index import PromptHelper
from llama_index import StorageContext, load_index_from_storage
from llama_index.evaluation import ResponseEvaluator
from llama_index.llms import OpenAI
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index.indices.query.query_transform import HyDEQueryTransform
from llama_index.query_engine.transform_query_engine import TransformQueryEngine
from langchain.chat_models import ChatOpenAI
from IPython.display import Markdown, display

import environ
import openai

In [100]:
# For now I use my key
env = environ.Env()
environ.Env.read_env()
API_KEY = env("OPENAI_API_KEY")
openai.api_key = API_KEY

# LLM

In [101]:
# LangChain
# selected_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [102]:
system_prompt = (
    """
    You are an expert on the German administration system and your job is to answer technical questions.
    Assume that all questions are related to the the provided context.
    Keep your answers based on facts, do not hallucinate information.
    """
)

selected_llm = OpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo",
    system_prompt=system_prompt
)

# Load vector store

In [103]:
embedding_path = "vector_db"
# rebuild storage context
storage_context_from_load = StorageContext.from_defaults(persist_dir=embedding_path)
# load index
index_loaded = load_index_from_storage(storage_context_from_load)

# Functions

In [104]:
# Evaluator
def eval_response(response):
    llm= LLMPredictor(llm=selected_llm)
    service_context_eval = ServiceContext.from_defaults(llm_predictor=llm)
    evaluator = ResponseEvaluator(service_context=service_context_eval)
    eval_result = evaluator.evaluate(response)
    if eval_result == "NO":
        print(str(eval_result))
        print("info not available")
        return None
    elif eval_result == "YES":
        display(Markdown(response.response))
        response_metadata_chat = dict()
        for i, source_node in enumerate(response.source_nodes):
            key_name = "ref_" + str(i)
            response_metadata_chat[key_name] = {
                "page": source_node.node.metadata["page_label"],
                "document":source_node.node.metadata["file_name"]
            }
        return response_metadata_chat
    else:
        print("something went wrong, try again!")
        return None

In [105]:
def print_response_info(user_query, response):
    print(f"Prompt:\n{user_query}")
    display(Markdown(f"Response:<br/>{response.response}"))
    print(f"Sources:")
    for node in response.source_nodes:
        print(f'Page: {node.node.metadata["page_label"]}, Document: {node.node.metadata["file_name"]}, Cosine Similarity: {node.score:.2f}')

# Define query and prompt template, folders and top_k

In [106]:
# user_query = "Is the number of children taken into account to calculate the unemployment benefit?"
user_query = "How can I calculate how much money I will get as unemployment benefit?"
# user_query = "What do you know about the city of Berlin?" # check for generic responses from GPT knowledge base
# user_query = "Wann muss ich mich melden wenn ich Arbeitslosengeld beantragen will?"

# Define prompt
template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Do not give me an answer if it is not mentioned in the context as a fact. \n"
    "Given this information, please provide me with an answer to the following question:\n{query_str}\n"
)
qa_template = Prompt(template)

folder_with_index = "vector_db"
number_top_results = 5 # Number of top results to return

# Query Engine

## Simple query engine

In [107]:
query_engine_baseline = index_loaded.as_query_engine(text_qa_template=qa_template, similarity_top_k=number_top_results)
response_baseline = query_engine_baseline.query(user_query)

print_response_info(user_query, response_baseline)

Prompt:
How can I calculate how much money I will get as unemployment benefit?


Response:<br/>To calculate how much money you will receive as unemployment benefit, you need to consider the following factors:

1. Bemessungsentgelt: The amount of unemployment benefit is based on your average income in the last year before becoming unemployed. This is known as the Bemessungsentgelt.

2. Lohnsteuerklasse: Your tax class also affects the calculation of unemployment benefit. The daily benefit rate is determined based on a standardized net income, which is reduced by social security contributions, solidarity surcharge, and income tax based on your tax class.

3. Number of children: If you have at least one child, the unemployment benefit rate may be increased to 67% of the standardized net income.

4. Additional income: If you have any additional income from part-time work or other sources, it will be taken into account and deducted from your unemployment benefit. There is a monthly exemption amount of 400 euros, and any income above this amount will be partially deducted from your benefit.

To get an accurate calculation of your unemployment benefit, you can use the self-calculation program provided by the employment agency. You can access this program on the website www.arbeitsagentur.de.

Sources:
Page: 9, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: 0.82
Page: 37, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.82
Page: 5, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: 0.81
Page: 43, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.81
Page: 23, Document: dok_ba015377.pdf, Cosine Similarity: 0.81


In [108]:
for uwdfujsn in response_baseline.source_nodes:
    print(uwdfujsn.node.metadata)

{'page_label': '9', 'file_name': 'merkblatt-nebeneinkommen_ba015792.pdf'}
{'page_label': '37', 'file_name': 'merkblatt-fuer-arbeitslose_ba036520.pdf'}
{'page_label': '5', 'file_name': 'merkblatt-nebeneinkommen_ba015792.pdf'}
{'page_label': '43', 'file_name': 'merkblatt-fuer-arbeitslose_ba036520.pdf'}
{'page_label': '23', 'file_name': 'dok_ba015377.pdf'}


In [109]:
response_text = response_baseline.response
response_metadata = dict()
response_metadata_message = f'There {len(response_baseline.metadata)} sources:'
for i, meta_data in enumerate(response_baseline.metadata):
    key_name = "ref_" + str(i)
    response_metadata[key_name] = {
        "page": response_baseline.metadata[meta_data]["page_label"],
        "document":response_baseline.metadata[meta_data]["file_name"]
    }
    response_metadata_message += "\n---" + key_name + f'Page {response_baseline.metadata[meta_data]["page_label"]} from file {response_baseline.metadata[meta_data]["file_name"]}'

In [110]:
response_baseline

Response(response='To calculate how much money you will receive as unemployment benefit, you need to consider the following factors:\n\n1. Bemessungsentgelt: The amount of unemployment benefit is based on your average income in the last year before becoming unemployed. This is known as the Bemessungsentgelt.\n\n2. Lohnsteuerklasse: Your tax class also affects the calculation of unemployment benefit. The daily benefit rate is determined based on a standardized net income, which is reduced by social security contributions, solidarity surcharge, and income tax based on your tax class.\n\n3. Number of children: If you have at least one child, the unemployment benefit rate may be increased to 67% of the standardized net income.\n\n4. Additional income: If you have any additional income from part-time work or other sources, it will be taken into account and deducted from your unemployment benefit. There is a monthly exemption amount of 400 euros, and any income above this amount will be part

In [111]:
response_metadata

{'ref_0': {'page': '9', 'document': 'merkblatt-nebeneinkommen_ba015792.pdf'},
 'ref_1': {'page': '37',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_2': {'page': '5', 'document': 'merkblatt-nebeneinkommen_ba015792.pdf'},
 'ref_3': {'page': '43',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_4': {'page': '23', 'document': 'dok_ba015377.pdf'}}

In [112]:
print(response_metadata_message)

There 5 sources:
---ref_0Page 9 from file merkblatt-nebeneinkommen_ba015792.pdf
---ref_1Page 37 from file merkblatt-fuer-arbeitslose_ba036520.pdf
---ref_2Page 5 from file merkblatt-nebeneinkommen_ba015792.pdf
---ref_3Page 43 from file merkblatt-fuer-arbeitslose_ba036520.pdf
---ref_4Page 23 from file dok_ba015377.pdf


In [113]:
eval_response(response_baseline)

To calculate how much money you will receive as unemployment benefit, you need to consider the following factors:

1. Bemessungsentgelt: The amount of unemployment benefit is based on your average income in the last year before becoming unemployed. This is known as the Bemessungsentgelt.

2. Lohnsteuerklasse: Your tax class also affects the calculation of unemployment benefit. The daily benefit rate is determined based on a standardized net income, which is reduced by social security contributions, solidarity surcharge, and income tax based on your tax class.

3. Number of children: If you have at least one child, the unemployment benefit rate may be increased to 67% of the standardized net income.

4. Additional income: If you have any additional income from part-time work or other sources, it will be taken into account and deducted from your unemployment benefit. There is a monthly exemption amount of 400 euros, and any income above this amount will be partially deducted from your benefit.

To get an accurate calculation of your unemployment benefit, you can use the self-calculation program provided by the employment agency. You can access this program on the website www.arbeitsagentur.de.

{'ref_0': {'page': '9', 'document': 'merkblatt-nebeneinkommen_ba015792.pdf'},
 'ref_1': {'page': '37',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_2': {'page': '5', 'document': 'merkblatt-nebeneinkommen_ba015792.pdf'},
 'ref_3': {'page': '43',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_4': {'page': '23', 'document': 'dok_ba015377.pdf'}}

In [114]:
print(response_baseline.get_formatted_sources())

> Source (Doc id: 26a2e343-6e3f-4259-a610-4b7567ad302f): 9erhalten, werden mit Ihrem Nettobetrag berücksich -
tigt und nach Abzug eines Freibetrages von 4...

> Source (Doc id: e93526b3-c61a-48ee-9671-db13277ee364): 4. Die Höhe der Leistung
384. Die Höhe der Leistung
Für die Höhe des Arbeitslosengeldes sind von ...

> Source (Doc id: 0fa8296e-e554-44ab-b518-b12e076a1aca): Wie wird Nebeneinkommen 
angerechnet?
Wenn Sie ein monatliches Nebeneinkommen in gleich  
bleiben...

> Source (Doc id: 94e9bd6f-e3b8-4c10-869b-bc42cfb74683): 4. Die Höhe der Leistung
44erster Tag der Arbeitslosigkeit) bereits Arbeitslosen ­
geld bezogen, ...

> Source (Doc id: d0920db7-26ff-4398-8b30-c5885c17b25a): 233 Finanzen
3 Finanzen
3.1 Informationen zum Arbeitslosengeld
Wenn Sie einen beruflichen Wiedere...


## Cosine Similarity Postprocessor

In [115]:
for node in response_baseline.source_nodes:
    cosine_similarity = node.score
    print(f"Cosine Similarity: {cosine_similarity}")

Cosine Similarity: 0.8201771222137203
Cosine Similarity: 0.8152714678838423
Cosine Similarity: 0.8108540073724564
Cosine Similarity: 0.8087323633109779
Cosine Similarity: 0.8082457646710791


In [116]:
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.821)
postprocessed_nodes = postprocessor.postprocess_nodes(response_baseline.source_nodes)

for node in postprocessed_nodes:
    print(f'Page: {node.node.metadata["page_label"]}, Document: {node.node.metadata["file_name"]}, Cosine Similarity: {node.score:.2f}')

## SVM

In [117]:
query_modes = [
    "svm",
    "linear_regression",
    "logistic_regression",
]

responses_modes = []

for query_mode in query_modes:
    query_engine_modes = index_loaded.as_query_engine(
        text_qa_template=qa_template,
        similarity_top_k=number_top_results,
        vector_store_query_mode=query_mode)

    response_modes = query_engine_modes.query(user_query)
    responses_modes.append(response_modes)

/home/daniele/Desktop/Projects/bureaubot/.venv_bureaubot/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/daniele/Desktop/Projects/bureaubot/.venv_bureaubot/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/daniele/Desktop/Projects/bureaubot/.venv_bureaubot/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [118]:
for query_mode, r in zip(query_modes, responses_modes):
    print(f"Query mode: {query_mode}\n")
    print_response_info(user_query, r)
    print("="*90)

Query mode: svm

Prompt:
How can I calculate how much money I will get as unemployment benefit?


Response:<br/>Based on the provided context information, there are two different scenarios mentioned for calculating the amount of unemployment benefit:

1. Nebeneinkommen (Side Income): If you have a monthly side income, the unemployment benefit will be reduced by the amount of the side income minus the exemption amount. The specific calculation can be found in the approval/changes notice (Bewilligungs-/Änderungsbescheid) provided by the employment agency.

2. Entlassungsentschädigung (Severance Pay): If you receive severance pay, a certain percentage of the total amount will be considered when calculating the suspension period (Ruhenszeitraum) for unemployment benefits. The specific calculation can be found in the table mentioned in section 4.4 of the document.

It is important to note that these calculations may vary depending on individual circumstances and the specific regulations of the employment agency. It is recommended to contact your local job center or employment agency for accurate and personalized information regarding the calculation of unemployment benefits.

Sources:
Page: 5, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: -0.66
Page: 9, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: -0.69
Page: 30, Document: merkblatt-17-entschaedigungen_ba015376.pdf, Cosine Similarity: -0.69
Page: 29, Document: merkblatt-17-entschaedigungen_ba015376.pdf, Cosine Similarity: -0.70
Page: 90, Document: merkblatt-algii_ba015397.pdf, Cosine Similarity: -0.71
Query mode: linear_regression

Prompt:
How can I calculate how much money I will get as unemployment benefit?


Response:<br/>Based on the provided context information, there are two different scenarios mentioned for calculating the amount of unemployment benefit:

1. Nebeneinkommen (Side Income): If you have a monthly side income, the unemployment benefit will be reduced by the amount of the side income minus the allowance. The specific calculation can be found in the approval/changes notice.

2. Entlassungsentschädigung (Severance Pay): If you receive severance pay, a certain percentage of the pay will be considered for the calculation of the unemployment benefit. The specific percentage and calculation can be found in the table mentioned in the document.

It is important to note that these calculations may vary depending on the specific regulations and guidelines of the relevant employment agency or job center. It is recommended to contact your local job center or employment agency for accurate and up-to-date information on how to calculate your unemployment benefit.

Sources:
Page: 5, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: -0.66
Page: 9, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: -0.69
Page: 30, Document: merkblatt-17-entschaedigungen_ba015376.pdf, Cosine Similarity: -0.69
Page: 29, Document: merkblatt-17-entschaedigungen_ba015376.pdf, Cosine Similarity: -0.70
Page: 90, Document: merkblatt-algii_ba015397.pdf, Cosine Similarity: -0.71
Query mode: logistic_regression

Prompt:
How can I calculate how much money I will get as unemployment benefit?


Response:<br/>Based on the provided context information, there are two different scenarios mentioned for calculating the amount of unemployment benefit:

1. Nebeneinkommen (Side Income): If you have a monthly side income, the unemployment benefit will be reduced by the amount of the income minus the allowance. The specific calculation can be found in the approval/changes notice.

2. Entlassungsentschädigung (Severance Pay): If you receive severance pay, a certain percentage of the pay will be considered for the calculation. The specific percentage and calculation can be found in the table mentioned in the document.

To calculate the exact amount of unemployment benefit, you would need to refer to the specific regulations and guidelines provided by the relevant authorities, such as the Agentur für Arbeit or Jobcenter. It is recommended to contact your local job center or employment agency for accurate information and assistance in calculating your specific unemployment benefit amount.

Sources:
Page: 5, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: -0.66
Page: 9, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: -0.69
Page: 30, Document: merkblatt-17-entschaedigungen_ba015376.pdf, Cosine Similarity: -0.69
Page: 29, Document: merkblatt-17-entschaedigungen_ba015376.pdf, Cosine Similarity: -0.70
Page: 90, Document: merkblatt-algii_ba015397.pdf, Cosine Similarity: -0.71


## Hypothetical document embeddings (HyDE)

In [119]:
query_engine_hyde = index_loaded.as_query_engine(text_qa_template=qa_template, similarity_top_k=number_top_results)

# "HyDEQueryTransform" generates a hypothetical document and use it for embedding lookup.
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine_hyde, hyde)
response_hyde = hyde_query_engine.query(user_query)

print_response_info(user_query, response_hyde)

Prompt:
How can I calculate how much money I will get as unemployment benefit?


Response:<br/>To calculate how much money you will receive as unemployment benefit, you need to consider the following factors mentioned in the provided context:

1. Bemessungsentgelt (Basis for Calculation): The amount of unemployment benefit is based on your average net income in the last year before becoming unemployed.

2. Freibetrag (Allowance): A monthly allowance of 400 euros is deducted from your net income before it is considered for the calculation of unemployment benefit.

3. Anrechnungsbetrag (Amount to be credited): After deducting the allowance, any remaining net income is credited towards your unemployment benefit. For example, if your net income is 500 euros and the allowance is 400 euros, 100 euros will be credited towards your unemployment benefit.

It's important to note that the specific calculation may vary depending on individual circumstances and additional factors. It is recommended to consult your local job center or employment agency for accurate information and assistance in calculating your unemployment benefit.

Sources:
Page: 9, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: 0.84
Page: 37, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.84
Page: 23, Document: dok_ba015377.pdf, Cosine Similarity: 0.84
Page: 39, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.84
Page: 4, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: 0.84


In [120]:
hyde

In [121]:
#In this example, HyDE improves output quality significantly, by hallucinating
# accurately, and thus improving the embedding quality, and final output.
query_bundle = hyde(user_query)
hyde_doc = query_bundle.embedding_strs[0]

print(f"Prompt:\n{user_query}\n\nResponse:")
display(Markdown(f"{hyde_doc}"))

Prompt:
How can I calculate how much money I will get as unemployment benefit?

Response:


To calculate how much money you will receive as unemployment benefits, there are several key details you need to consider. Firstly, you will need to determine your base period, which is typically the first four of the last five completed calendar quarters before you filed your claim. This period is used to calculate your earnings during that time.

Next, you will need to gather information on your total wages earned during the base period. This includes wages from all employers you worked for during that time, including part-time and temporary jobs. It is important to note that certain types of income, such as self-employment earnings or income from rental properties, may not be considered for unemployment benefit calculations.

Once you have your total wages, you will need to determine your average weekly wage (AWW). This is calculated by dividing your total wages earned during the base period by the number of weeks in that period. It is important to note that some states may have a maximum limit on the AWW, which can affect the final benefit amount.

After calculating your AWW, you will need to determine your weekly benefit amount (WBA). This is typically a percentage of your AWW, which varies by state. The percentage can range from around 40% to 60% of your AWW, with some states having a fixed maximum limit.

Additionally, some states may have a minimum and maximum weekly benefit amount, which can further impact the final calculation. It is important to check your state's specific guidelines to understand the exact percentage and limits that apply.

Finally, you will need to consider any additional factors that may affect your benefit amount. For example, if you have dependents, some states may provide an additional allowance for each dependent. Additionally, if you have any other sources of income while receiving unemployment benefits, such as part-time work, it may impact the amount you receive.

In summary, to calculate your unemployment benefit amount, you will need to determine your base period, calculate your average weekly wage, and apply the appropriate percentage to determine your weekly benefit amount. It is crucial to review your state's specific guidelines and consider any additional factors that may affect your benefit amount.

In [122]:
hyde_2 = HyDEQueryTransform(include_original=True)
query_bundle_2 = hyde_2(user_query)
hyde_doc_2 = query_bundle_2.embedding_strs[0]

print(f"Prompt:\n{user_query}\n\nResponse:")
display(Markdown(f"{hyde_doc_2}"))

Prompt:
How can I calculate how much money I will get as unemployment benefit?

Response:


To calculate how much money you will receive as unemployment benefits, there are several key details you need to consider. Firstly, you will need to determine your base period, which is typically the first four of the last five completed calendar quarters before you filed your claim. This period is used to calculate your earnings during that time.

Next, you will need to gather information on your total wages earned during the base period. This includes wages from all employers you worked for during that time, including full-time, part-time, and self-employment income. It is important to note that certain types of income, such as tips or commissions, may need to be reported separately.

Once you have your total wages, you will need to determine your average weekly wage (AWW). This is calculated by dividing your total wages earned during the base period by the number of weeks in that period. For example, if your base period is 52 weeks, you would divide your total wages by 52 to get your AWW.

After calculating your AWW, you will need to check your state's unemployment benefits program to determine the percentage of your AWW that will be provided as benefits. Each state has its own formula for calculating benefits, but it is typically a percentage of your AWW. For instance, if your state provides 50% of your AWW as benefits, you would multiply your AWW by 0.5 to determine your weekly benefit amount.

It is important to note that there are usually minimum and maximum benefit amounts set by each state. If your calculated benefit amount falls below the minimum, you will receive the minimum amount. Conversely, if your calculated benefit amount exceeds the maximum, you will receive the maximum amount.

Additionally, some states may have additional factors that can affect your benefit amount, such as dependents or additional income sources. It is crucial to review your state's specific guidelines to ensure you accurately calculate your unemployment benefits.

In summary, to calculate how much money you will receive as unemployment benefits, you need to determine your base period, calculate your total wages earned during that period, determine your average weekly wage, and then apply your state's formula to determine your weekly benefit amount. Remember to consider any additional factors or limitations set by your state's unemployment benefits program.

# Chat Engine

In [123]:
# Configure prompt parameters and initialise helper
max_input_size = 4096
num_output = 256
max_chunk_overlap = 0.2
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

# system_prompt = (
#     """
#     You are an expert on the German administration system and your job is to answer technical questions.
#     Assume that all questions are related to the the provided context.
#     Keep your answers based on facts, do not hallucinate information.
#     """
# )
# 
# llm= LLMPredictor(llm=OpenAI(
#     temperature=0,
#     model_name="gpt-3.5-turbo",
#     system_prompt=system_prompt
# ))
llm = LLMPredictor(selected_llm)
service_context = ServiceContext.from_defaults(llm_predictor=llm, prompt_helper=prompt_helper)

chat_engine = index_loaded.as_chat_engine(
    # service_context=service_context,
    chat_mode="context",
    # verbose=True
    similarity_top_k=3
)

In [124]:
response_chat = chat_engine.chat(user_query)

In [125]:
response_chat

AgentChatResponse(response="To calculate how much money you will receive as unemployment benefit, you need to consider the following factors:\n\n1. Bemessungsentgelt (Basis for Calculation): The amount of your unemployment benefit is based on your average taxable income in the last year before becoming unemployed. This is known as the Bemessungsentgelt.\n\n2. Lohnsteuerklasse (Tax Class): Your tax class is used to determine the deductions from your unemployment benefit. Different tax classes have different deduction rates.\n\n3. Kinderfreibetrag (Child Allowance): If you have children, you may be eligible for an increased rate of unemployment benefit. The child allowance is considered in the calculation.\n\nOnce you have these three pieces of information, you can use the following steps to calculate your unemployment benefit:\n\n1. Determine your Bemessungsentgelt: This is the average taxable income you earned in the last year before becoming unemployed.\n\n2. Determine your deductions

In [126]:
metadata_chat = eval_response(response_chat)

metadata_chat

To calculate how much money you will receive as unemployment benefit, you need to consider the following factors:

1. Bemessungsentgelt (Basis for Calculation): The amount of your unemployment benefit is based on your average taxable income in the last year before becoming unemployed. This is known as the Bemessungsentgelt.

2. Lohnsteuerklasse (Tax Class): Your tax class is used to determine the deductions from your unemployment benefit. Different tax classes have different deduction rates.

3. Kinderfreibetrag (Child Allowance): If you have children, you may be eligible for an increased rate of unemployment benefit. The child allowance is considered in the calculation.

Once you have these three pieces of information, you can use the following steps to calculate your unemployment benefit:

1. Determine your Bemessungsentgelt: This is the average taxable income you earned in the last year before becoming unemployed.

2. Determine your deductions: Based on your tax class, calculate the deductions from your Bemessungsentgelt.

3. Determine your Kinderfreibetrag: If you have children, calculate the additional amount you are eligible for based on the child allowance.

4. Calculate your daily unemployment benefit: Divide your Bemessungsentgelt (after deductions and Kinderfreibetrag) by 30 to get your daily unemployment benefit.

5. Calculate your monthly unemployment benefit: Multiply your daily unemployment benefit by the number of days in the month to get your monthly unemployment benefit.

It's important to note that these calculations may vary depending on the specific regulations and guidelines of your country or region. It is recommended to consult with your local employment agency or relevant authorities for accurate and up-to-date information on calculating unemployment benefits.

{'ref_0': {'page': '9', 'document': 'merkblatt-nebeneinkommen_ba015792.pdf'},
 'ref_1': {'page': '37',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_2': {'page': '5', 'document': 'merkblatt-nebeneinkommen_ba015792.pdf'}}

In [127]:
print(f"There {len(response_chat.source_nodes)} sources.")
for source_node in response_chat.source_nodes:
    print(f'Page {source_node.node.metadata["page_label"]} from file {source_node.node.metadata["file_name"]}')

There 3 sources.
Page 9 from file merkblatt-nebeneinkommen_ba015792.pdf
Page 37 from file merkblatt-fuer-arbeitslose_ba036520.pdf
Page 5 from file merkblatt-nebeneinkommen_ba015792.pdf


In [128]:
response_chat.response

"To calculate how much money you will receive as unemployment benefit, you need to consider the following factors:\n\n1. Bemessungsentgelt (Basis for Calculation): The amount of your unemployment benefit is based on your average taxable income in the last year before becoming unemployed. This is known as the Bemessungsentgelt.\n\n2. Lohnsteuerklasse (Tax Class): Your tax class is used to determine the deductions from your unemployment benefit. Different tax classes have different deduction rates.\n\n3. Kinderfreibetrag (Child Allowance): If you have children, you may be eligible for an increased rate of unemployment benefit. The child allowance is considered in the calculation.\n\nOnce you have these three pieces of information, you can use the following steps to calculate your unemployment benefit:\n\n1. Determine your Bemessungsentgelt: This is the average taxable income you earned in the last year before becoming unemployed.\n\n2. Determine your deductions: Based on your tax class, 

In [129]:
len(response_chat.source_nodes)

3

In [130]:
response_chat.source_nodes[0]

NodeWithScore(node=TextNode(id_='26a2e343-6e3f-4259-a610-4b7567ad302f', embedding=None, metadata={'page_label': '9', 'file_name': 'merkblatt-nebeneinkommen_ba015792.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e864b8b6-a752-4fe3-8bc8-9e112d83bfc7', node_type=None, metadata={'page_label': '9', 'file_name': 'merkblatt-nebeneinkommen_ba015792.pdf'}, hash='76c187b531eba95cd2f7d95a6eb6a6eff2fde5336100e8cae1869661bc1e520a')}, hash='76c187b531eba95cd2f7d95a6eb6a6eff2fde5336100e8cae1869661bc1e520a', text='9erhalten, werden mit Ihrem Nettobetrag berücksich -\ntigt und nach Abzug eines Freibetrages von 400 Euro \nmonatlich auf das Arbeitslosengeld angerechnet.\nBEISPIEL 4\nWegen der Teilnahme an einer Weiterbildung \nerhalten Sie von Ihrem Arbeitgeber eine monat -\nliche Vergütung von 500 Euro netto. Nach Ab -\nzug des Freibetrages von 400 Euro monatlich \nwerden 100 Euro monatlich auf das Arbeits-\

In [131]:
response_chat.source_nodes[0].node.metadata["page_label"]

'9'

# Compare responses

In [136]:
methods_list = ["Baseline", "SVM", "Linear regression", "Logistic regression", "HyDE", "HyDE doc", "Chat"]
responses_list = [
    response_baseline.response, responses_modes[0].response, responses_modes[1].response,
    responses_modes[2].response, response_hyde.response, hyde_doc, response_chat.response]

for m, r in zip(methods_list, responses_list):
    display(Markdown(f"{m}:<br/>{r}"))
    print("="*80)


Baseline:<br/>To calculate how much money you will receive as unemployment benefit, you need to consider the following factors:

1. Bemessungsentgelt: The amount of unemployment benefit is based on your average income in the last year before becoming unemployed. This is known as the Bemessungsentgelt.

2. Lohnsteuerklasse: Your tax class also affects the calculation of unemployment benefit. The daily benefit rate is determined based on a standardized net income, which is reduced by social security contributions, solidarity surcharge, and income tax based on your tax class.

3. Number of children: If you have at least one child, the unemployment benefit rate may be increased to 67% of the standardized net income.

4. Additional income: If you have any additional income from part-time work or other sources, it will be taken into account and deducted from your unemployment benefit. There is a monthly exemption amount of 400 euros, and any income above this amount will be partially deducted from your benefit.

To get an accurate calculation of your unemployment benefit, you can use the self-calculation program provided by the employment agency. You can access this program on the website www.arbeitsagentur.de.

SVM:<br/>Based on the provided context information, there are two different scenarios mentioned for calculating the amount of unemployment benefit:

1. Nebeneinkommen (Side Income): If you have a monthly side income, the unemployment benefit will be reduced by the amount of the side income minus the exemption amount. The specific calculation can be found in the approval/changes notice (Bewilligungs-/Änderungsbescheid) provided by the employment agency.

2. Entlassungsentschädigung (Severance Pay): If you receive severance pay, a certain percentage of the total amount will be considered when calculating the suspension period (Ruhenszeitraum) for unemployment benefits. The specific calculation can be found in the table mentioned in section 4.4 of the document.

It is important to note that these calculations may vary depending on individual circumstances and the specific regulations of the employment agency. It is recommended to contact your local job center or employment agency for accurate and personalized information regarding the calculation of unemployment benefits.

Linear regression:<br/>Based on the provided context information, there are two different scenarios mentioned for calculating the amount of unemployment benefit:

1. Nebeneinkommen (Side Income): If you have a monthly side income, the unemployment benefit will be reduced by the amount of the side income minus the allowance. The specific calculation can be found in the approval/changes notice.

2. Entlassungsentschädigung (Severance Pay): If you receive severance pay, a certain percentage of the pay will be considered for the calculation of the unemployment benefit. The specific percentage and calculation can be found in the table mentioned in the document.

It is important to note that these calculations may vary depending on the specific regulations and guidelines of the relevant employment agency or job center. It is recommended to contact your local job center or employment agency for accurate and up-to-date information on how to calculate your unemployment benefit.

Logistic regression:<br/>Based on the provided context information, there are two different scenarios mentioned for calculating the amount of unemployment benefit:

1. Nebeneinkommen (Side Income): If you have a monthly side income, the unemployment benefit will be reduced by the amount of the income minus the allowance. The specific calculation can be found in the approval/changes notice.

2. Entlassungsentschädigung (Severance Pay): If you receive severance pay, a certain percentage of the pay will be considered for the calculation. The specific percentage and calculation can be found in the table mentioned in the document.

To calculate the exact amount of unemployment benefit, you would need to refer to the specific regulations and guidelines provided by the relevant authorities, such as the Agentur für Arbeit or Jobcenter. It is recommended to contact your local job center or employment agency for accurate information and assistance in calculating your specific unemployment benefit amount.

HyDE:<br/>To calculate how much money you will receive as unemployment benefit, you need to consider the following factors mentioned in the provided context:

1. Bemessungsentgelt (Basis for Calculation): The amount of unemployment benefit is based on your average net income in the last year before becoming unemployed.

2. Freibetrag (Allowance): A monthly allowance of 400 euros is deducted from your net income before it is considered for the calculation of unemployment benefit.

3. Anrechnungsbetrag (Amount to be credited): After deducting the allowance, any remaining net income is credited towards your unemployment benefit. For example, if your net income is 500 euros and the allowance is 400 euros, 100 euros will be credited towards your unemployment benefit.

It's important to note that the specific calculation may vary depending on individual circumstances and additional factors. It is recommended to consult your local job center or employment agency for accurate information and assistance in calculating your unemployment benefit.

HyDE doc:<br/>To calculate how much money you will receive as unemployment benefits, there are several key details you need to consider. Firstly, you will need to determine your base period, which is typically the first four of the last five completed calendar quarters before you filed your claim. This period is used to calculate your earnings during that time.

Next, you will need to gather information on your total wages earned during the base period. This includes wages from all employers you worked for during that time, including part-time and temporary jobs. It is important to note that certain types of income, such as self-employment earnings or income from rental properties, may not be considered for unemployment benefit calculations.

Once you have your total wages, you will need to determine your average weekly wage (AWW). This is calculated by dividing your total wages earned during the base period by the number of weeks in that period. It is important to note that some states may have a maximum limit on the AWW, which can affect the final benefit amount.

After calculating your AWW, you will need to determine your weekly benefit amount (WBA). This is typically a percentage of your AWW, which varies by state. The percentage can range from around 40% to 60% of your AWW, with some states having a fixed maximum limit.

Additionally, some states may have a minimum and maximum weekly benefit amount, which can further impact the final calculation. It is important to check your state's specific guidelines to understand the exact percentage and limits that apply.

Finally, you will need to consider any additional factors that may affect your benefit amount. For example, if you have dependents, some states may provide an additional allowance for each dependent. Additionally, if you have any other sources of income while receiving unemployment benefits, such as part-time work, it may impact the amount you receive.

In summary, to calculate your unemployment benefit amount, you will need to determine your base period, calculate your average weekly wage, and apply the appropriate percentage to determine your weekly benefit amount. It is crucial to review your state's specific guidelines and consider any additional factors that may affect your benefit amount.

Chat:<br/>To calculate how much money you will receive as unemployment benefit, you need to consider the following factors:

1. Bemessungsentgelt (Basis for Calculation): The amount of your unemployment benefit is based on your average taxable income in the last year before becoming unemployed. This is known as the Bemessungsentgelt.

2. Lohnsteuerklasse (Tax Class): Your tax class is used to determine the deductions from your unemployment benefit. Different tax classes have different deduction rates.

3. Kinderfreibetrag (Child Allowance): If you have children, you may be eligible for an increased rate of unemployment benefit. The child allowance is considered in the calculation.

Once you have these three pieces of information, you can use the following steps to calculate your unemployment benefit:

1. Determine your Bemessungsentgelt: This is the average taxable income you earned in the last year before becoming unemployed.

2. Determine your deductions: Based on your tax class, calculate the deductions from your Bemessungsentgelt.

3. Determine your Kinderfreibetrag: If you have children, calculate the additional amount you are eligible for based on the child allowance.

4. Calculate your daily unemployment benefit: Divide your Bemessungsentgelt (after deductions and Kinderfreibetrag) by 30 to get your daily unemployment benefit.

5. Calculate your monthly unemployment benefit: Multiply your daily unemployment benefit by the number of days in the month to get your monthly unemployment benefit.

It's important to note that these calculations may vary depending on the specific regulations and guidelines of your country or region. It is recommended to consult with your local employment agency or relevant authorities for accurate and up-to-date information on calculating unemployment benefits.